In [1]:
import pandas as pd
import json
from collections import defaultdict

from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure, curdoc, show
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.palettes import Category10

output_notebook()

Loading BokehJS ...

In [2]:
# Ruta relativa al archivo JSON
ruta_json = "../Jsonl/course-creaaa1/course-creaaa1-limpio.json"

In [3]:
# Ejemplo de uso:
codigo_a_nombre = {
    "video_intro": "vKq2NotGPJQ",
    "LR_1_Video1_Semana1": "U3cK1QMIIEQ",
    "LR_1_Video2_Semana1": "9aNQZ9dKXRY",
    "LR_1_Video3_Semana1": "lsNxh-lSpCY",
    "LR_1_Video4_Semana1": "C3LnEvN0qZ0",
    "LR_1_Video5_Semana1": "vbpbkQE5K_Q",
    "LR_1_Video6_Semana1": "zCFa0xjGXGQ",
    "LR_1_Video7_Semana1": "qlS7ShZfb-c",
    "LR_1_Video8_Semana1": "8cKRb9CKtxk",
    "LR_1_Video9_Semana1": "WyrfIZ6VBcM",
    "LR_1_Video10_Semana1": "NgUhK3rw1IE",
    "LR_1_Video11_Semana1": "ttP0EyzSbbo",
    "LR_1_Video12_Semana1": "Vy4FWDyjZo4",
    "LR_2_Video1_Semana1": "leg7NPlfNf0",
    "LR_2_Video2_Semana1": "avTMbQWrFgM",
    "LR_2_Video3_Semana1": "cNoUwGM1DQs",
    "LR_2_Video4_Semana1": "6Mst559v-Uc",
    "LR_2_Video5_Semana1": "CNQpefXv5DY",
    "LR_2_Video_Semana1": "6W1_fBZFqns",
    "LR_1_Video1_Semana2": "o5VwDVJ7N3Q",
    "LR_1_Video2_Semana2": "LluqYlh2xg4",
    "LR_1_Video3_Semana2": "eE658thjDj8",
    "LR_1_Video4_Semana2": "QbEpClHzTeM",
    "LR_1_Video5_Semana2": "MCG0or2ULB4",
    "LR_1_Video6_Semana2": "ol-vGTdHBNU",
    "LR_1_Video7_Semana2": "WTXS0IMQ3Ss",
    "LR_1_Video8_Semana2": "9kqXmM3b3wc",
    "VS1_Video1_Semana2": "_zQHV3vCGpA",
    "VS2_Video2_Semana2": "RropOrUc2AE",
    "Video1_Semana3": "VGHSSIUyFhI",
    "Video1_Semana4": "kyGRuJXaboU",
}

# Lista de interacciones relacionadas con videos
interacciones_video = ["play_video", "pause_video", "seek_video", "stop_video"]

In [4]:
# Leer el archivo JSON
with open(ruta_json, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convertir a DataFrame
data = pd.DataFrame(data)

In [5]:
# Función para contar interacciones
def contar_interacciones(data, interacciones):
    conteos = defaultdict(lambda: defaultdict(int))
    for _, row in data.iterrows():
        if row.get("name") in interacciones:
            try:
                evento_json = json.loads(row.get("event", '{}'))
                codigo_video = evento_json.get("code", "")
                if codigo_video:
                    conteos[row.get("name")][codigo_video] += 1
            except json.JSONDecodeError as e:
                print(f"Error al decodificar línea: {e}")
    return conteos

In [6]:
# Convertir los códigos de video a nombres
def convertir_codigos_a_nombres(conteos, codigo_a_nombre):
    conteos_con_nombres = defaultdict(dict)
    for interaccion, videos in conteos.items():
        for codigo, conteo in videos.items():
            nombre_video = codigo_a_nombre.get(codigo, codigo)
            conteos_con_nombres[interaccion][nombre_video] = conteo
    return conteos_con_nombres

In [7]:
# Obtener los conteos y convertir los códigos
conteos = contar_interacciones(data, interacciones_video)
conteos_con_nombres = convertir_codigos_a_nombres(conteos, codigo_a_nombre)

In [19]:
# Función para preparar los datos para la gráfica
def preparar_datos(conteos_con_nombres, interaccion_seleccionada):
    if interaccion_seleccionada == "Todas las interacciones":
        data = {"videos": list(codigo_a_nombre.values())}
        for tipo, videos in conteos_con_nombres.items():
            data[tipo] = [videos.get(nombre, 0) for nombre in codigo_a_nombre.values()]
    else:
        data = {
            "videos": list(codigo_a_nombre.values()),
            "reproducciones": [
                conteos_con_nombres.get(interaccion_seleccionada, {}).get(nombre, 0)
                for nombre in codigo_a_nombre.values()
            ],
        }
    return ColumnDataSource(data=data)

In [18]:
# Callback para actualizar la gráfica
def actualizar_grafica(attr, old, new):
    nueva_fuente = preparar_datos(conteos_con_nombres, select.value)
    source.data = nueva_fuente.data
    p.renderers = []
    if select.value == "Todas las interacciones":
        p.vbar_stack(
            list(conteos_con_nombres.keys()),
            x="videos",
            source=source,
            width=0.8,
            color=Category10[len(conteos_con_nombres)],
            legend_label=list(conteos_con_nombres.keys()),
        )
        p.legend.click_policy = "mute"
    else:
        p.vbar(
            x="videos",
            top="reproducciones",
            width=0.8,
            source=source,
            fill_color="skyblue",
            line_color="black",
        )

In [21]:
# Configuración inicial
interaccion_inicial = "Todas las interacciones"
source = preparar_datos(conteos_con_nombres, interaccion_inicial)

In [29]:
# Crear gráfica
p = figure(
    x_range=list(codigo_a_nombre.values()),
    title="Interacciones por Video",
    x_axis_label="Videos",
    y_axis_label="Reproducciones",
    width=800,
    height=400,
    tools="pan,box_zoom,wheel_zoom,save,reset",
    toolbar_location="right",
)
p.xaxis.major_label_orientation = 0.785

# p.vbar_stack(
#    list(conteos_con_nombres.keys()),
#    x="videos",
#    source=source,
#    width=0.8,
#    color=Category10[len(conteos_con_nombres)],
#    legend_label=list(conteos_con_nombres.keys()),
#)
#p.legend.click_policy = "mute"

# Crear el widget Select
#select = Select(
#    title="Seleccionar interacción",
#    value=interaccion_inicial,
#    options=["Todas las interacciones"] + list(conteos_con_nombres.keys()),
#)
#select.on_change("value", actualizar_grafica)

# Layout final
#layout_final = row(select, p)
#curdoc().add_root(layout_final)
#show(layout_final)

if interaccion_inicial == "Todas las interacciones":
    p.vbar_stack(
        list(conteos_con_nombres.keys()),
        x="videos",
        source=source,
        width=0.8,
        color=Category10[len(conteos_con_nombres)],
        legend_label=list(conteos_con_nombres.keys()),
    )
else:
    p.vbar(
        x="videos",
        top="reproducciones",
        width=0.8,
        source=source,
        fill_color="skyblue",
        line_color="black",
    )
    
# Mostrar directamente el gráfico
show(p)